# Estimator 객체의 개념
Scikit-learn에서 머신러닝 모델, 데이터 전처리기 등을 통칭하는 가장 기본적인 객체로 모든 Estimator는 일관된 인터페이스를 가진다.<br>
즉, 모델을 학습시킬 때는 `fit()`을, 예측할 때는 `predict()`를, 데이터를 변환할 때는 `transform()`을 사용한다. <br>
어떤 모델(선형 회귀, 결정 트리)이나 전처리기(데이터 스케일러, 인코더)를 사용하든 이 일관된 패턴이 유지되므로 매우 직관적이고 편리한다.

- 일관성 덕분에 우리는 특정 알고리즘의 세부 구현을 알 필요 없이, 마치 레고 블록을 조립하듯 다양한 모델과 전처리기를 쉽게 교체하고 조합할 수 있다. 이는 복잡한 머신러닝 파이프라인을 구축하는 데 필수적인 요소이다.

- 실제 사용 사례:
  - 모델: `LinearRegression()`, `LogisticRegression()`, `KMeans()`
  - 전처리기: `StandardScaler()`, `OneHotEncoder()`, `SimpleImputer()`

- `fit()`: 모델 학습
  - fit(X, y) 메서드는 Estimator 객체를 학습시키는 역할을 한다. 지도 학습 모델의 경우 입력 데이터 X(특성 행렬)와 정답 데이터 y(타겟 벡터)를 사용하여 모델 내부 파라미터(예: 선형 회귀의 계수)를 최적화한다.
  - 입력 데이터 구조:
    - 특성 행렬(X): [n_samples, n_features] 형태의 2차원 배열(NumPy ndarray 또는 Pandas DataFrame). 각 행은 하나의 샘플, 각 열은 하나의 특성을 나타낸다.
    - 타겟 벡터(y): [n_samples] 형태의 1차원 배열. X의 각 행에 해당하는 정답 데이터이다.

- `predict()`: 새로운 데이터 예측
  - `predict(X_new)` 메서드는 학습된 모델을 사용하여 새로운, 보지 못했던 데이터 X_new에 대한 예측값을 반환한다. 이 메서드는 학습 과정에서 사용된 타겟 y를 필요로 하지 않다.
  - `predict()`는 항상 새로운 데이터에 대해서만 사용해야 한다. 이미 학습에 사용된 데이터로 예측하면 모델의 일반화 성능을 제대로 평가할 수 없다.

- transform(): 데이터 변환
  - `transform(X)` 메서드는 전처리기(Transformer) 객체가 입력 데이터를 변환하는 데 사용된다. 예를 들어, StandardScaler는 fit() 단계에서 학습한 평균과 표준편차를 이용해 transform() 단계에서 데이터를 표준화한다.

- fit_transform(): 학습과 변환 동시 수행
  - `fit_transform(X, y)` 메서드는 fit()과 transform()을 순서대로 호출하는 편리한 기능이다. 주로 전처리 단계의 학습 데이터에 사용되어, 한 번의 메서드 호출로 변환에 필요한 파라미터를 학습하고 데이터를 즉시 변환한다.
  - 주의사항 및 팁: **fit_transform()**은 학습 데이터에만 사용해야 한다. 테스트 데이터에는 **fit()**을 호출하지 않고, **transform()**만 사용하여 학습 데이터의 통계치를 그대로 적용해야 데이터 유출(Data Leakage)을 방지할 수 있다.


## StandardScaler
사이킷런에서 StandardScaler는 데이터 전처리(preprocessing)에 사용되는 변환기(transformer)의 한 종류이다.

StandardScaler는 데이터셋의 특성(feature)들을 **표준화(standardization)**하는 데 사용한다. 표준화는 각 특성의 평균을 0, 표준편차를 1로 조정하여 데이터의 스케일을 통일하는 작업이다.

- 작동원리<br>
StandardScaler는 다음의 수학 공식을 각 특성(X)에 적용하여 변환한다.

$$X_{\text{new}} = \frac{X - \mu}{\sigma}$$

- X_new : 변환된 새로운 값
- X: 원래 값
- μ: 특성의 평균(mean)
- σ: 특성의 표준편차(standard deviation)

이러한 표준화 과정을 통해 데이터의 분포가 **정규 분포(normal distribution)**와 유사하게 되며, 이는 많은 머신러닝 알고리즘의 성능 향상에 도움을 준다.


> 왜 StandardScaler를 사용해야 하는가? <br>
대부분의 머신러닝 알고리즘은 특성들의 스케일이 다르면 특정 특성에만 과도하게 가중치를 부여할 수 있다.<br> 예를 들어, 나이(10-100)와 연봉(2,000만-1억)이라는 두 특성이 있을 때, 연봉의 스케일이 훨씬 크므로 알고리즘이 연봉에 더 큰 영향을 받게 된다.<br>
StandardScaler는 이러한 문제를 해결하여 모든 특성이 모델에 공정하게 기여하도록 돕는다. 특히 **SVM, 로지스틱 회귀, 신경망, K-평균 군집화(K-Means clustering)**와 같이 거리 기반의 계산을 사용하는 알고리즘에서 매우 중요하다.

#### 목적: fit(), transform(), fit_transform()의 차이를 이해하고 사용법을 익힌다.

1. 데이터 준비: 붓꽃 데이터셋에서 특성 데이터만 불러와 NumPy 배열로 변환한다.

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 붓꽃 데이터셋 로드
iris = load_iris()
X = iris.data # 특성 데이터만 사용 (타겟 y는 필요 없음)

print("원본 데이터 X.shape:", X.shape) # 150개의 데이터와 4개의 특징
print("원본 데이터 X (일부):\n", X[:5])

원본 데이터 X.shape: (150, 4)
원본 데이터 X (일부):
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


2. StandardScaler 객체 생성 및 fit_transform() 사용: <br>훈련 데이터에 대해 학습과 변환을 동시에 수행한다.

In [4]:
# StandardScaler 객체 생성 (Estimator 객체)
scaler = StandardScaler()

# 훈련 데이터에 fit_transform() 적용
# 각 특성별로 연산을 수행한다.
# - 현재 4개의 특성
X_scaled = scaler.fit_transform(X)
print("\nfit_transform() 결과 X_scaled (일부):\n", X_scaled[:5])
# std : 표준편차
print("fit_transform() 적용 후 평균: {:.4f}, 표준편차: {:.4f}".format(X_scaled.mean(), X_scaled.std()))


fit_transform() 결과 X_scaled (일부):
 [[-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
 [-1.14301691 -0.13197948 -1.34022653 -1.3154443 ]
 [-1.38535265  0.32841405 -1.39706395 -1.3154443 ]
 [-1.50652052  0.09821729 -1.2833891  -1.3154443 ]
 [-1.02184904  1.24920112 -1.34022653 -1.3154443 ]]
fit_transform() 적용 후 평균: -0.0000, 표준편차: 1.0000


3. StandardScaler 객체 생성 및 fit() 사용: <br>훈련 데이터에 대해 학습만 진행하기 때문에 추론은 따로 진행해야 한다.

In [8]:
# StandardScaler 객체 생성 (Estimator 객체)
scaler_fit = StandardScaler()

# 훈련 데이터 X에 fit() 메서드를 적용하여 평균과 표준편차를 학습
scaler_fit.fit(X)

# 학습된 scaler 객체를 사용하여 X 데이터를 실제로 변환
X_fit_scaled = scaler_fit.transform(X)

print("\nfit_transform() 결과 X_scaled (일부):\n", X_fit_scaled[:5])
print("fit_transform() 적용 후 평균: {:.4f}, 표준편차: {:.4f}".format(X_fit_scaled.mean(), X_fit_scaled.std()))


fit_transform() 결과 X_scaled (일부):
 [[-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
 [-1.14301691 -0.13197948 -1.34022653 -1.3154443 ]
 [-1.38535265  0.32841405 -1.39706395 -1.3154443 ]
 [-1.50652052  0.09821729 -1.2833891  -1.3154443 ]
 [-1.02184904  1.24920112 -1.34022653 -1.3154443 ]]
fit_transform() 적용 후 평균: -0.0000, 표준편차: 1.0000


> 결과 확인:<br> fit_transform()을 사용한 데이터의 평균과 표준편차는 0과 1에 매우 가깝게 나타난다.<br> fit()과 transform()을 분리해서 사용했을 때, 훈련 데이터는 마찬가지로 0과 1에 가깝지만,<br> 테스트 데이터는 훈련 데이터의 통계량으로 변환되었기 때문에 정확히 0과 1이 아닐 수 있다.

### 첫 번째 분류 모델 만들기 - K-NN (K-Nearest Neighbors)
fit()과 predict() 메서드를 사용하여 간단한 분류 모델을 구축하고 예측을 수행한다.


#### K-최근접 이웃 (K-NN)
**K-최근접 이웃(K-Nearest Neighbors)**의 약자로, 분류(classification)와 회귀(regression)에 사용되는 머신러닝 알고리즘 중 하나이다. <br>주로 분류 문제에 사용한다.

<img src="https://drive.google.com/uc?id=1cCj90f8O1bDYS1pH4uJ1qfeFcReoF8Vh" width=400 />

> 작동 원리<br>
> K-NN은 데이터 포인트를 분류할 때, 그 데이터와 가장 가까운 K개의 이웃을 찾고, 그 이웃들의 클래스(분류)를 기준으로 새로운 데이터의 클래스를 결정한다.
> - 거리 계산: 새로운 데이터 포인트와 기존 훈련 데이터 포인트들 사이의 거리를 계산한다.
> - K개 이웃 선택: 계산된 거리를 기준으로 가장 가까운 K개의 이웃을 선택한다.
> - 다수결 투표: K개의 이웃 중에서 가장 많은 비율을 차지하는 클래스로 새로운 데이터 포인트를 분류한다.
> - 예를 들어, K=3일 때 3개의 가장 가까운 이웃 중 2개가 '고양이'이고 1개가 '강아지'라면, 새로운 데이터는 '고양이'로 분류된다.

#### Scikit-learn lib 알아보기
1. `sklearn.datasets`
  - 이 라이브러리는 머신러닝 학습에 사용할 수 있는 예제 데이터셋을 제공한다.
  - `load_iris`: 붓꽃(Iris) 데이터셋을 불러오는 함수이다. 이 데이터셋은 3가지 종류의 붓꽃(setosa, versicolor, virginica)과 각 붓꽃의 꽃받침, 꽃잎 길이를 포함하고 있다. 머신러닝 모델의 분류 성능 테스트에 자주 사용된다
2. `sklearn.model_selection`
  - 이 라이브러리는 모델을 평가하고 하이퍼파라미터 튜닝을 위해 데이터를 나누는 기능을 제공한다.
  - `train_test_split`: 데이터셋을 **훈련 데이터(training data)**와 **테스트 데이터(test data)**로 분리하는 함수이다. 모델은 훈련 데이터로 학습하고, 테스트 데이터로 성능을 평가한다. 이를 통해 모델이 새로운 데이터에 얼마나 잘 일반화되는지 확인할 수 있다.
3. `sklearn.neighbors`
  - 이 라이브러리는 K-NN(K-최근접 이웃) 알고리즘을 포함한 이웃 기반의 머신러닝 모델을 제공한다.
  - `KNeighborsClassifier`: K-NN 분류 모델이다. 이 모델은 새로운 데이터 포인트와 가장 가까운 K개의 훈련 데이터 이웃을 찾아 다수결로 분류를 결정한다.

4. `sklearn.metrics`
  - 이 라이브러리는 머신러닝 모델의 성능을 평가하는 다양한 지표를 제공한다.
  - `accuracy_score` : **정확도(Accuracy)**를 계산하는 함수이다. 모델이 예측한 값과 실제 값(정답)을 비교하여 얼마나 정확하게 예측했는지 백분율로 나타낸다. 예를 들어, 100개 중 90개를 맞췄다면 정확도는 0.9가 된다.




1. 데이터 준비: 붓꽃 데이터셋을 다시 불러와 특성과 타겟을 분리하고, 훈련/테스트 데이터로 나눈다.

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 데이터 로드 및 분할
iris = load_iris()
X = iris.data # 데이터
y = iris.target # 정답

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

display(X[:5])
display(y)

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

2. 모델 학습: KNeighborsClassifier Estimator를 생성하고, fit() 메서드로 모델을 학습시킨다.

In [18]:
# K-NN 모델 객체 생성 (Estimator)
knn = KNeighborsClassifier(n_neighbors=5)
# n_neighbors : K-NN 알고리즘에서 사용할 이웃의 수(K)

# 훈련 데이터로 모델 학습
knn.fit(X_train, y_train)

KNeighborsClassifier()

3. 예측 및 평가: 학습된 모델을 사용하여 테스트 데이터에 대한 예측을 수행하고, accuracy_score로 성능을 평가한다.

In [19]:
# 테스트 데이터로 예측
y_pred = knn.predict(X_test)

# 예측 결과 확인
print("실제 값 (y_test):\n", y_test)
print("\n예측 값 (y_pred):\n", y_pred)

# 정확도(Accuracy) 계산
accuracy = accuracy_score(y_test, y_pred)
print("\n예측 정확도: {:.4f}".format(accuracy))

실제 값 (y_test):
 [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]

예측 값 (y_pred):
 [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]

예측 정확도: 1.0000
